# Open Farming Hackdays 2020: Open Data Farming

## Get and parse from KoBoToolbox via API 

In [1]:
import requests
import pandas as pd
from geopy.geocoders import Nominatim

import geopandas as gpd
from shapely.geometry import LineString, shape

from plotnine import *

# init locator module    
locator = Nominatim(user_agent="myGeocoder")

In [2]:
# set correct access data
# login via browser and get your access token by calling the following URL
# https://kf.kobotoolbox.org/token/?format=json
token = '###'

# see URL of project website
base_url = "http://kf.kobotoolbox.org/api/v2/"
project_id = "aiTfpho7SNDDYUQBerqqSj"
assets_project = f"assets/{project_id}/data.json"

url_project = base_url + assets_project
print("Getting data from:", url_project)

# parse data
r = requests.get(url_project, headers={'Authorization': f'Token {token}'})
data = r.json()

Getting data from: http://kf.kobotoolbox.org/api/v2/assets/aiTfpho7SNDDYUQBerqqSj/data.json


In [3]:
def lookup_geographic_information(event):
    geo_info = {}

    # create lookup key
    coord_lookup = f"{event['x']}, {event['y']}"
    location = locator.reverse(coord_lookup).raw
    
    
    lookup_data = [("municipality", 'city'), ("municipality", 'village'), ('canton', 'state'), ('country', 'country')]
    for key, val in lookup_data:
        try:
            geo_info[key] = location['address'][val]
        except KeyError:
            pass
    
    return geo_info
        

def parse_single_event(row):
    
    lookup_data = [("date", 'start'), ('geolocation', 'geolocation'), ('disease', 'disease')]
    
    event = {}

    for key, val in lookup_data:
        try: 
            event[key] = row[val]
        except KeyError:
            pass
    
    try:
        loc = event['geolocation'].split()
        event['x'], event['y']  = loc[0], loc[1]
        geo_info = lookup_geographic_information(event)
        event = {**event, **geo_info}
    except KeyError:
        pass
        
    return event 
    
def get_dataset(data):
    rows = []

    for row in data['results']:
        event = parse_single_event(row)
        rows.append(event)
    
    df = pd.DataFrame(rows)   
    
    df['x'] = pd.to_numeric(df.x)
    df['y'] = pd.to_numeric(df.y)
    
    return df
    
df = get_dataset(data)
df

,date,geolocation,disease,x,y,municipality,canton,country
0,2020-09-18T21:47:12.542+02:00,47.375461 8.555248 0 2614,echter_mehltau,47.375461,8.555248,Zürich,Zürich,Switzerland
1,2020-09-18T21:47:39.885+02:00,47.375461 8.555248 0 2614,echter_mehltau,47.375461,8.555248,Zürich,Zürich,Switzerland
2,2020-09-18T21:50:24.141+02:00,47.375461 8.555248 0 2614,echter_mehltau,47.375461,8.555248,Zürich,Zürich,Switzerland
3,2020-09-18T21:50:39.246+02:00,47.390883 7.645324 0 0,echter_mehltau,47.390883,7.645324,Bretzwil,Basel-Landschaft,Switzerland
4,2020-09-18T21:51:13.272+02:00,NaN,echter_mehltau,NaN,NaN,NaN,NaN,NaN
5,2020-09-18T21:51:28.229+02:00,46.24083 7.301877 0 0,echter_mehltau,46.240830,7.301877,Conthey,Valais/Wallis,Switzerland


## Case-Study: Merge field data with a soil map from the Swiss Goverment

In [4]:
# file needs to located in the same directory using its orignal name

resource_folder = 'data/'
f_soil = resource_folder + "Bodeneignungskarte_Landwirtschaft_D/Bodeneignungskarte der Schweiz LV95 Shape/Bodeneignungskarte_LV95.shp"
df_soil = gpd.read_file(f_soil)


# convert the geolocation into the EPSG:4326 format
df_soil["geometry"] = df_soil.geometry.to_crs(epsg=4326)

df_soil

,AREA,PERIMETER,BEK200_,BEK200_ID,INT_CODE,CODE,Farbe,Shape_Leng,KU_CODE,Eignungsei,...,Bodentyp,GRUNDIGKEI,SKELETT,WASSERSPEI,NAHRSTOFF,WASSERDURC,VERNASS,Shape_Le_1,Shape_Area,geometry
0,1.553976e+06,6583.278008,2,7,15,A5,7,6583.278003,4,"Futterbau, Grossviehweide: +; Getreideb.: +/-",...,"gleyic, eutric, dystric Cambisol; eutric Gleysol",5.0,2.0,5.0,5.0,3.0,2.0,6583.272340,1.553974e+06,"POLYGON ((8.61017 47.79435, 8.60957 47.79470, ..."
1,9.795158e+05,5873.426517,3,8,12,A2,16,5873.426506,5,Kleinviehweide: +; Jungviehw.: +/-; Grossviehw...,...,"Rendzina; dystric calcaric, eutric Regosol",2.0,5.0,2.0,3.0,5.0,1.0,5873.425736,9.795159e+05,"POLYGON ((8.61017 47.79435, 8.60952 47.79391, ..."
2,5.885888e+05,4310.725235,4,10,82,H2,4,4310.725302,1,Futterbau: ++; Getreideb.: +; Hackfruchtb.: +/-,...,"eutric Fluvisol; gleyic, eutric, calcaric Camb...",5.0,3.0,5.0,5.0,4.0,3.0,4310.727788,5.885890e+05,"POLYGON ((8.66578 47.78935, 8.66244 47.78904, ..."
3,3.863492e+05,2786.847525,5,15,16,A6,9,2786.847546,4,"Naturf-b.: +; Kunstf-b., Getreideb.: +/-; Gros...",...,"eutric, gleyic Cambisol; mollic, humic Gleysol",5.0,2.0,4.0,5.0,2.0,2.0,2786.846805,3.863495e+05,"POLYGON ((8.57425 47.79524, 8.57507 47.79565, ..."
4,9.721035e+05,8291.883397,6,16,13,A3,15,8291.883449,5,Kleinviehw.: ++; Jungviehw.: +; Grossviehw.: +/-,...,"eutric, dystric Regosol; Rendzina (Jura), calc...",3.0,4.0,3.0,4.0,3.0,2.0,8291.880299,9.721044e+05,"POLYGON ((8.57425 47.79524, 8.57433 47.79442, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11072,1.198235e+06,6840.219049,11064,9952,221,V1,18,6840.219050,5,Kleinviehweide: +/-,...,"Lithosol; eutric, calcaric Regosol; Ranker; Re...",2.0,5.0,1.0,1.0,6.0,1.0,6840.269801,1.198257e+06,"POLYGON ((9.87491 46.40355, 9.87514 46.40429, ..."
11073,4.505514e+06,19989.113775,11065,9971,231,W1,18,19989.113797,5,Kleinviehweide: +/-,...,"Lithosol; eutric, calcaric Regosol; Ranker; Re...",2.0,5.0,1.0,1.0,6.0,1.0,19989.237387,4.505527e+06,"POLYGON ((9.76530 46.42449, 9.77501 46.41787, ..."
11074,1.111675e+06,7087.331431,11066,10010,235,W5,17,7087.331416,5,Kleinviehweide: +; Jungviehweide: +/-,...,Eutric Regosol; Rendzina (Jura); Ranker; Lithosol,2.0,4.0,2.0,2.0,6.0,1.0,7087.427887,1.111706e+06,"POLYGON ((9.76648 46.42000, 9.76846 46.41929, ..."
11075,2.174662e+06,10503.339494,11067,10065,237,W7,17,10503.339721,5,Kleinviehweide: +; Jungviehweide: +/-,...,Eutric Regosol; Rendzina (Jura); Ranker; Lithosol,2.0,4.0,2.0,2.0,6.0,1.0,10503.439109,2.174701e+06,"POLYGON ((9.74945 46.41343, 9.75005 46.41366, ..."


In [5]:
# show and simplify original categories 
df_soil['field_type'] = df_soil.Eignungsei.str.replace("(.*),.*", r"\1")
df_soil['field_type'] =  df_soil.field_type.str.replace(".*vieh.*", "Viehweide")
df_soil['field_type'] =  df_soil.field_type.str.replace(".*Acker.*", "Acker")
df_soil['field_type'] =  df_soil.field_type.str.replace(".*Getreide.*", "Getreide")
df_soil['field_type'] =  df_soil.field_type.str.replace(".*[Ff]utterbau.*", "Futterbau")
df_soil['field_type'] =  df_soil.field_type.str.replace(".*Naturf-b.*", "Futterbau")
df_soil['field_type'] =  df_soil.field_type.str.replace(".*Siedlungsgebiet.*", "keine Agrarfläche")
df_soil['field_type'] =  df_soil.field_type.str.replace(".*Seen.*", "Seen")

print(df_soil.Eignungsei.unique())

['Futterbau, Grossviehweide: +; Getreideb.: +/-'
 'Kleinviehweide: +; Jungviehw.: +/-; Grossviehw.: -'
 'Futterbau: ++; Getreideb.: +; Hackfruchtb.: +/-'
 'Naturf-b.: +; Kunstf-b., Getreideb.: +/-; Grossvi*'
 'Kleinviehw.: ++; Jungviehw.: +; Grossviehw.: +/-'
 'Jungviehw.: ++; Naturfutterb.: +; Grossviehw.: +/-'
 'Getreidebau, Naturfutterb., Grossviehweide: +/-'
 'Getreidebau, Futterbau: ++, Hackfruchtbau: +/++'
 'Siedlungsgebiete, Fels, Gletscher'
 'Naturfutterbau: +; Kunstfutterbau: +/-'
 'Acker-, Naturfutterbau: +; Kunstfutterbau: +/-'
 'Futterbau: +/++; Ackerbau: +' 'Seen, Enklaven'
 'Naturfutterbau: +; Kunstfutterb., Getreideb.: +/-'
 'Kleinviehweide: +; Futterbau, Jungviehweide: +/-'
 'Grossviehweide: ++; Futterbau: +; Getreideb.: +/-'
 'Kleinviehweide: +; Jungviehweide: +/-' 'Kleinviehweide: +/-'
 'Grossviehweide: +; Futterbau: +/-'
 'Jungviehweide: +; Grossviehw., Naturfutterb.: +/-']


In [6]:
def coord_to_list(geom):
    coords = list(geom.exterior.coords)
    return (coords)

def linestring_to_list(line):
    return list(shape(line).coords)

# extract coordinates from polygon data type
df_soil['coordinates'] = df_soil['geometry'].apply(coord_to_list)

In [7]:
# explode dataset on field type and coordinates
df_soil['id'] = df_soil.index
df_soil = df_soil[['id', "field_type", 'coordinates']].explode("coordinates")

# parse latitude and longitude values
df_soil['long'] = df_soil['coordinates'].apply(lambda x: x[0])
df_soil['lat'] = df_soil['coordinates'].apply(lambda x: x[1])

In [10]:
# plot spread of all diseases 

disease = 'echter_mehltau'

p = (ggplot() +
  geom_polygon(df_soil, aes(x = "long", y = "lat", group = "id", fill = "field_type"), alpha = 0.7) +
  geom_count(df[df.disease == disease], aes(x = "y", y= 'x',), color='black', alpha= 0.8) +
 scale_size_area(max_size = 5, breaks=range(1,5)) + 
 theme_void() +
   labs(title = "Verbreitung Echter Mehltau ",
       fill = "Gebietsnutzung", size='Anzahl Meldungen') +
 guides(
    color = guide_colorbar(order = 0),
    fill = guide_legend(order = 1)
  )
)


p.save('plots/agriculture_disease_spread.png', dpi=300)

/home/alex/.local/lib/python3.8/site-packages/plotnine/ggplot.py:727: PlotnineWarning: Saving 6.4 x 4.8 in image.
  warn("Saving {0} x {1} {2} image.".format(
/home/alex/.local/lib/python3.8/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: plots/agriculture_disease_spread.png
  warn('Filename: {}'.format(filename), PlotnineWarning)
/home/alex/.local/lib/python3.8/site-packages/plotnine/utils.py:1246: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if pdtypes.is_categorical(arr):
/home/alex/.local/lib/python3.8/site-packages/plotnine/layer.py:360: PlotnineWarning: stat_sum : Removed 1 rows containing non-finite values.
  data = self.stat.compute_layer(data, params, layout)
